# Namibia's Master Facility List: Obtaining the data via API

A master facility list (MFL) contains the full complement of health facilities of a country, including those in public as well as private sector. The list should include the geographic location of each facility along with other attributes such as the type of facility (e.g. hospital, clinic, pharmacy, or other), services rendered, infrastructure available, and more. The World Health Organisation published a [guide for the development and strengthening of MFLs](https://www.who.int/healthinfo/country_monitoring_evaluation/mfl/en/).

A number of African countries have started to develop master health facility lists. Namibia, amongst others, have taken the extra steps to make the data available online via an application programming interface (API).

As part of work around MFLs in Africa, we accessed the [Namibian MFL](https://www.who.int/healthinfo/country_monitoring_evaluation/mfl/en/) API using the Python code below. This data will be used in further comparisons with other MFLs from Africa.

In [2]:
# Import packages to access the data
import json
import requests
import pandas as pd
# Needed to ensure we don't access the API too often
from time import sleep
import numpy as np

## API access via Python

### SSL trouble

Despite the fact that the API can be accessed directly from a web browser without errors, it was not possible to follow the direct route to download the data in Python or R. The code below was suggested by [Peter van Heusden](https://twitter.com/pvanheus?lang=en) to circumvent the challenges experienced due to troublesome SSL configuration on the side of the API.

### Data structure

It is necessary to make two calls to the API to access detailed information about each facility listed in the MFL. The first call gets `facilities.json` that contains a list of facilities and their location but most importantly, allows us to access the ID of each facility. In a subsequent API call, we can use the facility ID to get a JSON file containing detailed information about each facility, including services and infrastructure available.

In the code below we will download a JSON file as Python dictionary. The JSON contains several layers of nesting. A single key `locations` contains a list of dictionaries with keys `id`, `parent_id`, `name`, `long_name`, `contact_person`, `phone_number`, `alt_phone_number`, `address`, `catchment_population`, `point_x`, `point_y`, `location_type`, `location_ownership`, `children`. In this dictionary the `id` refers to the ID of the region.

Each `locations`.`children` contains another dictionary called `children` that contains keys `id`, `parent_id`, `long_name`, `contact_person`, `phone_number`, `alt_phone_number`, `address`, `catchment_population`, `point_x`, `point_y`, `location_type`, `location_ownership`, `children`.

In the first API call we aim to get a list of `locations.children.id` since this is the actual ID of the facility itself.

In [30]:
# Code created by Peter van Heusden (helped to overcome SSL problem)

# This URL gives access to rudimentary information about each facility including the ID
url = 'https://mfl.mhss.gov.na/api/facilities.json'
response = requests.get(url, verify=False)
# Data is a python dictionary
data = response.json()

/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [31]:
# This function is needed to gain access to the third layer dictionary where we find the 
# facility IDs as explained in the text above
# The code was developed by Brett Mullins and kindly shared on the Internet in June 2019
# https://bcmullins.github.io/parsing-json-python/


def extract_element_from_json(obj, path):
    '''
    Extracts an element from a nested dictionary or
    a list of nested dictionaries along a specified path.
    If the input is a dictionary, a list is returned.
    If the input is a list of dictionary, a list of lists is returned.
    obj - list or dict - input dictionary or list of dictionaries
    path - list - list of strings that form the path to the desired element
    '''
    def extract(obj, path, ind, arr):
        '''
            Extracts an element from a nested dictionary
            along a specified path and returns a list.
            obj - dict - input dictionary
            path - list - list of strings that form the JSON path
            ind - int - starting index
            arr - list - output list
        '''
        key = path[ind]
        if ind + 1 < len(path):
            if isinstance(obj, dict):
                if key in obj.keys():
                    extract(obj.get(key), path, ind + 1, arr)
                else:
                    arr.append(None)
            elif isinstance(obj, list):
                if not obj:
                    arr.append(None)
                else:
                    for item in obj:
                        extract(item, path, ind, arr)
            else:
                arr.append(None)
        if ind + 1 == len(path):
            if isinstance(obj, list):
                if not obj:
                    arr.append(None)
                else:
                    for item in obj:
                        arr.append(item.get(key, None))
            elif isinstance(obj, dict):
                arr.append(obj.get(key, None))
            else:
                arr.append(None)
        return arr
    if isinstance(obj, dict):
        return extract(obj, path, 0, [])
    elif isinstance(obj, list):
        outer_arr = []
        for item in obj:
            outer_arr.append(extract(item, path, 0, []))
        return outer_arr

In [32]:
# Create a list with all facility IDs

facility_ids = extract_element_from_json(data, ["locations", "children", "children", "id"])

### Getting the actual facility details

Now that we have a list of facility IDs, we can do a second API call that references each facility and gets us the following information: `id`, `name`, `long_name`, `contact_person`, `phone_number`, `alt_phone_number`, `address`, `catchment_population`, `point_x`, `point_y` with sub_dictionaries: 

- `parent_location` with keys `id`, `name`
- `location_type` with keys `name`
- `location_ownership` with keys `name`
- `infrastructures` with sub_dictionaries each with keys `id`, `name`
- `services` with sub_dictionaries each with keys `id`, `name`

`infrastructures` can include Ambulances, Beds, Electricity, Running Water, Toilets, Computers, Vehicles, Enrolled Nurses, Registered Nurses, Doctors, etc.

`services` can include HIV Testing Services, General Clinical Service, Expanded Programme on Immunizations, Preventing Mother to Child Transmission Services, Viral Load Testing, Sexual Transmitted Infections, Anti Retroviral Therapy, Ante Natal Clinic Services, Family Planning Services, and more.

In [113]:
# Create emtpy list to append individual dataframes - 1 dataframe per facility
facility_list = []

# Iterate over all facility IDs identified in previous step
for id in facility_ids:
    # Construct API call from facility ID identified in earlier step
    facility_url = "https://mfl.mhss.gov.na/api/facilities/id/" + str(id) + ".json"
    
    # Request data from API
    facility_response = requests.get(facility_url, verify=False)
    
    # Convert to dict object
    facility_data = facility_response.json()
    
    # Create list of attributes for the facility that will be converted into dataframe in next step
    temp_list = {'name': extract_element_from_json(facility_data, 
                                                         ["facility_details", "name"]),
                 'id': extract_element_from_json(facility_data, 
                                                         ["facility_details", "id"]),
                 'long_name': extract_element_from_json(facility_data, 
                                                         ["facility_details", "long_name"]),
                 'contact_person': extract_element_from_json(facility_data, 
                                                         ["facility_details", "contact_person"]),
                 'phone_number': extract_element_from_json(facility_data, 
                                                         ["facility_details", "phone_number"]),
                 'alt_phone_number': extract_element_from_json(facility_data, 
                                                         ["facility_details", "address"]),
                 'catchment_population': extract_element_from_json(facility_data, 
                                                         ["facility_details", "catchment_population"]),
                 'point_x': extract_element_from_json(facility_data, 
                                                         ["facility_details", "point_x"]),
                 'point_y': extract_element_from_json(facility_data, 
                                                         ["facility_details", "point_y"]),
                 'parent_location_id': extract_element_from_json(facility_data, 
                                                         ["facility_details", "parent_location", "id"]),
                 'parent_location_name': extract_element_from_json(facility_data, 
                                                         ["facility_details", "parent_location", "name"]),
                 'location_type_name': extract_element_from_json(facility_data, 
                                                         ["facility_details", "location_type", "name"]),
                 'location_ownership_name': extract_element_from_json(facility_data, 
                                                         ["facility_details", "location_ownership", "name"])
                }
    
    # Add service and infrastructure elements

    # Get list of infrastructures
    infr_list = extract_element_from_json(facility_data, ["facility_details", "infrastructures"])

    # The infrastructure list is actually stored as list of lists.
    # To access the list elements one has to access the list within the list hence infr_list[0]
    # Some facilities don't have infrastructure listed
    if len(infr_list[0]) > 0:
        # Extract a list of infrastructure IDs
        infr_list_ids = extract_element_from_json(facility_data, ["facility_details", "infrastructures", "id"])
        # Extract a list of infrastructure names
        infr_list_names = extract_element_from_json(facility_data, ["facility_details", "infrastructures", "name"])
        # Add infrastructure information to temp_list
        # ID list is a list of numbers - to use join we first have to convert each number to a string using the map function
        temp_list['infrastructure_ids'] = ",".join(map(str,infr_list_ids))
        temp_list['infrastructure_names'] = ",".join(infr_list_names)    
    else:
        # Add empty strings if no infrastructure was listed
        temp_list['infrastructure_ids'] = ""
        temp_list['infrastructure_names'] = ""
        
    # Get list of services
    serv_list = extract_element_from_json(facility_data, ["facility_details", "services"])
    
    # The services list is actually stored as list of lists.
    # To access the list elements one has to access the list within the list hence serv_list[0]
    # Some facilities don't have services listed
    if len(serv_list[0]) > 0:
        # Extract a list of infrastructure IDs
        serv_list_ids = extract_element_from_json(facility_data, ["facility_details", "services", "id"])
        print(serv_list_ids, type(serv_list_ids))
        # Extract a list of infrastructure names
        serv_list_names = extract_element_from_json(facility_data, ["facility_details", "services", "name"])
        
        # Add services information to temp_list
        # ID list is a list of numbers - to use join we first have to convert each number to a string using the map function
        temp_list['service_ids'] = ",".join(map(str,serv_list_ids))
        temp_list['service_names']= ",".join(serv_list_names)
    else:
        # Add empty strings if no services were listed
        temp_list['service_ids'] = ""
        temp_list['service_names'] = ""
            
    
    # Convert contents from 'facility_details' dict to dataframe otherwise each dataframe consist of a single column
    fac_df = pd.DataFrame(temp_list)
    
    # Add facility type column based on facility type observed in 'name'
    # Use case_when type implementation found in R from 
    # https://stackoverflow.com/questions/54653356/case-when-function-from-r-to-python
    # These facility type choices was obtained from the drop-down list at https://mfl.mhss.gov.na/location-manager/locations
    # We looked at all datasets that could be downloaded via the API but none contained a facility type other than "Facility"
    # We did however notice that the facility type was always encoded in the name of the facility so am extracting it from there
    
    fac_type_choices = ["Hospital", "Intermediate Hospital", "Health Centre", "Clinic", "Rehab Centre", "Laboratory", 
                      "Prison", "Regional Health Office", "District Health Office", "CBART", "Mobile Van"]
    

    # Because the Namibian site differentiates between Hospital and Intermediate Hospital (but not other types of 
    # hospitals despite specifying it in the naming convention), we have to find all names that contain the word
    # "Hospital" but not the word "Intermediate Hospital"
    # We use the "look around" functionality in regular expressions - https://www.rexegg.com/regex-lookarounds.html
    # ?>! translates to look backwards and see that the pattern following does not exist before the target pattern
    # In our case we are saying "See that you don't find Intermediate before Hospital"
    fac_type_conditions = [(bool(re.search("^.*(?<!Intermediate)Hospital", fac_df['name'][0]))),
                              (bool(re.search("Intermediate Hospital", fac_df['name'][0]))),
                              (bool(re.search("Health Centre", fac_df['name'][0]))),
                              (bool(re.search("Clinic", fac_df['name'][0]))),
                              (bool(re.search("Rehab Centre", fac_df['name'][0]))),
                              (bool(re.search("Laboratory", fac_df['name'][0]))),
                              (bool(re.search("Prison", fac_df['name'][0]))),
                              (bool(re.search("Regional Health Office", fac_df['name'][0]))),
                              (bool(re.search("District Health Office", fac_df['name'][0]))),
                              (bool(re.search("CBART", fac_df['name'][0]))),
                              (bool(re.search("Mobile Van", fac_df['name'][0]))),
                              ]
    # Add the column using the conditions and choices specified above
    fac_df["facility_type"] = np.select(fac_type_conditions, fac_type_choices)
    
    # Add new dataframe to list of dataframes
    facility_list.append(fac_df)
   
    # Wait 1 second between requests to API (there are no guidelines about using the API)
    sleep(1)

    
# Convert list of dataframes to single dataframe
all_facilities = pd.concat(facility_list)

/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1,2,5,6,7,9,11,13,14,16,17,19,20
[1, 2, 3, 8, 31, 32, 36, 38, 39, 41, 52, 53] <class 'list'>


/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


5,6,7,9,11,16,17,19
[1, 2, 3, 8, 31, 32, 36, 38, 39, 41, 52, 53] <class 'list'>


/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


...

/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


2,5,6,7,9,10,11,13,16,17
[1, 3, 8, 31, 32, 38, 39, 49, 52, 53] <class 'list'>


/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


1,2,4,5,6,7,8,9,10,11,12,14,16
[1, 2, 3, 4, 8, 31, 32, 37, 38, 39, 40, 41, 49, 52, 53] <class 'list'>


/home/talarify/anaconda3/lib/python3.6/site-packages/urllib3/connectionpool.py:847: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [114]:
# Check that we actually have a row for each facility
len(all_facilities) == len(facility_ids)

True

In [115]:
#Check the data formatting - make sure this is something sensible
all_facilities.head()

,name,id,long_name,contact_person,phone_number,alt_phone_number,catchment_population,point_x,point_y,parent_location_id,parent_location_name,location_type_name,location_ownership_name,infrastructure_ids,infrastructure_names,service_ids,service_names,facility_type
0,Zambezi Regional Health Office,11981,None,None,None,None,None,-17.4994,24.2788,10598,Katima Mulilo District,Facility,Public_MoHSS,,,,,Regional Health Office
0,Sibbinda Health Centre,10131,None,,None,,4111,-17.7851,23.8212,10598,Katima Mulilo District,Facility,Public_MoHSS,"1,2,5,6,7,9,11,13,14,16,17,19,20","Ambulances,Beds,Electricity,Running Water,Heal...","1,2,3,8,31,32,36,38,39,41,52,53","HIV Testing Services,General Clinical Service,...",Health Centre
0,Sesheke Clinic,10130,None,,None,,5156,-17.7518,23.3975,10598,Katima Mulilo District,Facility,Public_MoHSS,"5,6,7,9,11,16,17,19","Electricity,Running Water,Health Extension Wor...","1,2,3,8,31,32,36,38,39,41,52,53","HIV Testing Services,General Clinical Service,...",Clinic
0,Schuckmansburg Clinic,10129,None,,None,,1844,-17.5487,24.815,10598,Katima Mulilo District,Facility,Public_MoHSS,"4,5,6,7,9,11,14,16,17,19","Mortuary,Electricity,Running Water,Health Exte...","1,2,3,8,31,32,36,38,39,41,52,53","HIV Testing Services,General Clinical Service,...",Clinic
0,Sangwali Health Centre,10128,None,,None,,4791,-18.2649,23.6375,10598,Katima Mulilo District,Facility,Public_MoHSS,"1,2,5,6,7,9,11,13,14,16,17,19,20","Ambulances,Beds,Electricity,Running Water,Heal...","1,2,3,8,31,32,36,38,39,41,52,53","HIV Testing Services,General Clinical Service,...",Health Centre


In [7]:
# Some facility types cannot easily be deduced from the name as above and gets "0" assigned in the step above
# Change "0" to "Other" to allow some scripts in afrhihealthsites not to bomb out (requires type so cannot have empty string)

all_facilities["facility_type"] = all_facilities["facility_type"].replace("0", "Other")

In [10]:
# Write dataframe to CSV for further analysis 
all_facilities.to_csv('../data/raw_data/namibia.csv', encoding='utf-8', index = False)